In [1]:
pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install vaderSentiment


  Obtaining dependency information for vaderSentiment from https://files.pythonhosted.org/packages/76/fc/310e16254683c1ed35eeb97386986d6c00bc29df17ce280aed64d55537e9/vaderSentiment-3.3.2-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/126.0 kB ? eta -:--:--
   --- ------------------------------------ 10.2/126.0 kB ? eta -:--:--
   ------ -------------------------------- 20.5/126.0 kB 217.9 kB/s eta 0:00:01
   ------------------------- ------------- 81.9/126.0 kB 657.6 kB/s eta 0:00:01
   -------------------------------------- 126.0/126.0 kB 826.5 kB/s eta 0:00:00


In [3]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from tabulate import tabulate

In [4]:
data = [
    ("I love this product, it's amazing!", 'positive'),
    ("This product is terrible, I hate it.", 'negative'),
    ("It's okay, not bad but not great either.", 'neutral'),
    ("Best product ever, highly recommended!", 'positive'),
    ("I'm really disappointed with the quality.", 'negative'),
    ("So-so product, nothing special about it.", 'neutral'),
    ("The customer service was excellent!", 'positive'),
    ("I wasted my money on this useless product.", 'negative'),
    ("It's not the worst, but certainly not the best.", 'neutral'),
    ("I can't live without this product, it's a lifesaver!", 'positive'),
    ("The product arrived damaged and unusable.", 'negative'),
    ("It's average, neither good nor bad.", 'neutral'),
    ("Highly disappointed with the purchase.", 'negative'),
    ("The product exceeded my expectations.", 'positive'),
    ("It's just okay, nothing extraordinary.", 'neutral'),
    ("This product is excellent, it exceeded all my expectations!", 'positive'),
    ("I regret purchasing this product, it's a waste of money.", 'negative'),
    ("It's neither good nor bad, just average.", 'neutral'),
    ("Outstanding customer service, highly recommended!", 'positive'),
    ("I'm very disappointed with the quality of this item.", 'negative'),
    ("It's not the best product, but it gets the job done.", 'neutral'),
    ("This product is a game-changer, I can't imagine life without it!", 'positive'),
    ("I received a defective product, very dissatisfied.", 'negative'),
    ("It's neither great nor terrible, just okay.", 'neutral'),
    ("Fantastic product, I would buy it again in a heartbeat!", 'positive'),
    ("Avoid this product at all costs, complete waste of money.", 'negative'),
    ("It's decent, but nothing extraordinary.", 'neutral'),
    ("Impressive quality, exceeded my expectations!", 'positive'),
    ("I'm very unhappy with this purchase, total disappointment.", 'negative'),
    ("It's neither amazing nor terrible, somewhere in between.", 'neutral')
]

In [5]:
#insert the data into a table + extra columns for additional information
table_data = [["Text", "Actual Label", "TextBlob Polarity", "TextBlob Sentiment", "VADER Compound", "VADER Sentiment"]]

In [6]:
# do the looping to insert the data by using Textblob and do sentiment analysis
for text, actual_label in data:
    # TextBlob
    blob = TextBlob(text)
    tb_polarity = blob.sentiment.polarity
    
    # Determine label based on polarity score from TextBlob
    if tb_polarity > 0:
        tb_label = 'positive'
    elif tb_polarity < 0:
        tb_label = 'negative'
    else:
        tb_label = 'neutral'

    # VADER
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(text)
    vader_compound = vs['compound']

    # Determine label based on compound score from VADER
    if vader_compound > 0.05:
        vader_label = 'positive'
    elif vader_compound < -0.05:
        vader_label = 'negative'
    else:
        vader_label = 'neutral'

    table_data.append([text, actual_label, tb_polarity, tb_label, vader_compound, vader_label])


In [7]:
print(tabulate(table_data, headers="firstrow", tablefmt="plain"))

Text                                                              Actual Label      TextBlob Polarity  TextBlob Sentiment      VADER Compound  VADER Sentiment
I love this product, it's amazing!                                positive                  0.625      positive                        0.8516  positive
This product is terrible, I hate it.                              negative                 -0.9        negative                       -0.7783  negative
It's okay, not bad but not great either.                          neutral                   0.15       positive                       -0.4707  negative
Best product ever, highly recommended!                            positive                  0.6        positive                        0.7639  positive
I'm really disappointed with the quality.                         negative                 -0.75       negative                       -0.5256  negative
So-so product, nothing special about it.                          neutral        

### DO COMPARISON WITH THE ACTUAL EXPRESSION (without ML)

In [10]:
#import classification report
from sklearn.metrics import classification_report

# Calculate classification report for TextBlob
tb_classification_report = classification_report([label for _, label, _, tb_label, _, _ in table_data[1:]], [tb_label for _, _, _, tb_label, _, _ in table_data[1:]], target_names=['negative', 'neutral', 'positive'])

# Calculate classification report for VADER
vader_classification_report = classification_report([label for _, label, _, _, _, vader_label in table_data[1:]], [vader_label for _, _, _, _, _, vader_label in table_data[1:]], target_names=['negative', 'neutral', 'positive'])

# Print classification report for TextBlob
print("\nClassification Report for TextBlob:")
print(tb_classification_report)

# Print classification report for VADER
print("\nClassification Report for VADER:")
print(vader_classification_report)



Classification Report for TextBlob:
              precision    recall  f1-score   support

    negative       0.67      0.80      0.73        10
     neutral       0.00      0.00      0.00        10
    positive       0.53      0.80      0.64        10

    accuracy                           0.53        30
   macro avg       0.40      0.53      0.46        30
weighted avg       0.40      0.53      0.46        30


Classification Report for VADER:
              precision    recall  f1-score   support

    negative       0.56      1.00      0.71        10
     neutral       0.33      0.10      0.15        10
    positive       0.89      0.80      0.84        10

    accuracy                           0.63        30
   macro avg       0.59      0.63      0.57        30
weighted avg       0.59      0.63      0.57        30



### DO COMPARISON WITH ML

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [12]:
# take the original data then split them into TEST and TRAINING DATA
# Split data into training and testing sets
texts = [text for text, _ in data]
labels = [label for _, label in data]
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.4, random_state=42)

In [13]:
# Extract features (bag of words representation)
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [14]:
# Initialize classifiers
nb_classifier = MultinomialNB()
svm_classifier = SVC(kernel='linear')

In [16]:
# Train classifiers
nb_classifier.fit(X_train, y_train)
svm_classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [17]:
# Predict sentiment using classifiers
for text, actual_label in zip(X_test, y_test):
    # Predict sentiment using Naive Bayes
    nb_prediction = nb_classifier.predict(text)[0]

    # Predict sentiment using SVM
    svm_prediction = svm_classifier.predict(text)[0]


In [18]:
# Calculate classification report for Naive Bayes
nb_classification_report = classification_report(y_test, nb_classifier.predict(X_test), target_names=['negative', 'neutral', 'positive'])

# Calculate classification report for SVM
svm_classification_report = classification_report(y_test, svm_classifier.predict(X_test), target_names=['negative', 'neutral', 'positive'])

# Print classification report for Naive Bayes
print("\nClassification Report for Naive Bayes:")
print(nb_classification_report)

# Print classification report for SVM
print("\nClassification Report for SVM:")
print(svm_classification_report)



Classification Report for Naive Bayes:
              precision    recall  f1-score   support

    negative       0.80      1.00      0.89         4
     neutral       0.75      1.00      0.86         3
    positive       1.00      0.60      0.75         5

    accuracy                           0.83        12
   macro avg       0.85      0.87      0.83        12
weighted avg       0.87      0.83      0.82        12


Classification Report for SVM:
              precision    recall  f1-score   support

    negative       0.75      0.75      0.75         4
     neutral       0.75      1.00      0.86         3
    positive       0.75      0.60      0.67         5

    accuracy                           0.75        12
   macro avg       0.75      0.78      0.76        12
weighted avg       0.75      0.75      0.74        12

